In [ ]:
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

In [ ]:
load_dotenv()

# Configure API key
try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Missing GOOGLE_API_KEY environment variable")
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    raise


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# โหลดไฟล์ PDF
pdf_path = "./That Time I Got Reincarnated as a Slime LN 21(Eng).pdf" 
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"โหลดเอกสารทั้งหมด {len(documents)} หน้า")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)
split_docs = text_splitter.split_documents(documents)
print(f"แบ่งข้อความออกเป็นทั้งหมด {len(split_docs)} chunks")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains.base import Chain

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-04-17",
    temperature=1.2,
    timeout=None,
    max_retries=2,
)

prompt_template = """
Translate the following English text to Thai. 
Please maintain the original tone, style, and literary nuances of a novel.
If the word is the name of the characters do not translate to Thai!!.

ENGLISH TEXT:
"{text}"

THAI TRANSLATION:
"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["text"],
)

translation_chain = prompt | llm

In [ ]:
import time

translated_parts = []
total_chunks = len(split_docs)

for i, doc in enumerate(split_docs):
    try:
        
        input_text = doc.page_content
        
        result = translation_chain.invoke({"text": input_text})
        
        translated_text = result.content
        
        translated_parts.append(translated_text)
        
        print(f"แปล Chunk {i+1}/{total_chunks} เสร็จสิ้น")
        
        time.sleep(4.1) 

    except Exception as e:
        print(f"เกิดข้อผิดพลาดที่ Chunk {i+1}: {e}")
        translated_parts.append(f"[--- ERROR TRANSLATING CHUNK {i+1} ---]")

final_translation = "\n".join(translated_parts)

# บันทึกผลลัพธ์ลงไฟล์
with open("translated_novel_th.txt_1", "w", encoding="utf-8") as f:
    f.write(final_translation)

print("\nการแปลเสร็จสมบูรณ์! บันทึกไฟล์แล้วที่ translated_novel_th.txt")